In [13]:
# Transformers installation
! pip install transformers datasets evaluate accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
#加载数据集，还是imdb，影评集，这里是通过hugging face进行加载的
from datasets import load_dataset

imdb = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Then take a look at an example:

In [5]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [6]:
#AutoTokenizer是一个 “自动匹配” 工具，能根据预训练模型的名称，自动加载该模型对应的分词器
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")#DistilBERT基础版模型，“uncased”表示分词器不区分大小写

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
#预处理函数，用于对文本进行分词并截断序列，使其不超过DistilBERT的最大输入长度
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
#将预处理函数应用于整个数据集
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
#创建一个样本批次，将句子动态填充至批次中最长句子的长度
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)#动态填充

In [14]:
#Evaluate库可以快速加载评估方法
import evaluate

accuracy = evaluate.load("accuracy")#这里我们使用准确率作为评估指标

In [15]:
#创建一个函数，把结果和标签都传入进去，用于计算准确率
import numpy as np


def compute_metrics(eval_pred):#eval_pred是模型在评估集上的预测结果与真实标签的组合
    predictions, labels = eval_pred#获取预测值和真实值
    predictions = np.argmax(predictions, axis=1)#将模型的概率分布转换为具体的类别预测，axis=1表示沿第1个轴取最大值的索引
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
#创建预期id到标签的映射，情感分类，所以一共就肯定和否定两种
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
#使用AutoModelForSequenceClassification类加载DistilBERT，同时指定预期的标签数量和标签映射
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#训练参数配置
training_args = TrainingArguments(
    output_dir="my_awesome_model",#模型保存路径
    learning_rate=2e-5,
    per_device_train_batch_size=16,#训练批次大小
    per_device_eval_batch_size=16,#评估批次大小
    num_train_epochs=2,
    weight_decay=0.01,#权重衰减
    eval_strategy="epoch",#评估策略
    save_strategy="epoch",#保存策略
    load_best_model_at_end=True,#训练结束后加载性能最好的模型
    push_to_hub=False,#没登陆huggingface，没办法保存
    report_to="none"#关闭所有实验跟踪工具，wandb需要登录的，这是一个专门用于机器学习实验跟踪、可视化和协作的工具
)

#Trainer初始化
trainer = Trainer(
    model=model,
    args=training_args,#训练参数
    train_dataset=tokenized_imdb["train"],#训练用数据集
    eval_dataset=tokenized_imdb["test"],#测试用数据集
    processing_class=tokenizer,#分词器
    data_collator=data_collator,#动态填充的函数
    compute_metrics=compute_metrics,#计算准确率的函数
)

trainer.train()#开始微调

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.223400,0.207348,0.919960
2,0.145000,0.234173,0.931120


TrainOutput(global_step=3126, training_loss=0.2047465922965198, metrics={'train_runtime': 3012.1942, 'train_samples_per_second': 16.599, 'train_steps_per_second': 1.038, 'total_flos': 6556904415524352.0, 'train_loss': 0.2047465922965198, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

In [19]:
#测试：
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [20]:
#使用pipeline工具快速加载微调后的模型
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(text)

config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9994940757751465}]

In [21]:
#一下为手动复现上面的测试
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")#分词
inputs = tokenizer(text, return_tensors="pt")

In [23]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [24]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'